# Daily Crime Bulletin - Howard County, MD  
Scraped crime data from the publicly available website of the Howard County, MD.

##### Imports and constants

In [ ]:
# imports
import requests
import pandas as pd
import re
import numpy as np
import math
from bs4 import BeautifulSoup

# initialize variables
url = 'https://www.howardcountymd.gov/police/newsroom'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

# compiled regex
cat_regex = re.compile(r'<p><strong><u>(?P<cat>[a-zA-Z \/-]*)<\/u><br\/>')
city_regex = re.compile(r'(?P<city>[a-zA-z ]*),\s\d{5}')
zip_regex = re.compile(r'[a-zA-z ]*,\s(?P<zip>\d{5}):')
date_regex = re.compile(r'(?P<date>\w+\.?\s\d+-?\d*)')
time_regex = re.compile(r'\s(?P<time>\d{1,2}:\d{1,2}\s*[a.mp]*)')
street_regex = re.compile(r':\s?<\/strong>(?P<street>.*),\s?(March|Feb.|February)?')
notes_time_regex = re.compile(r':\d{1,2}\s*[a.mp]*<br\/>\s(.*)<')
notes_time_br_p_regex = re.compile(r'[a.mp]*<br\/>\s([\w\s]*)</p>')
notes_time_br_br_regex = re.compile(r'[a.mp]*<br\/>\s([\w\s,.]*)<br/>')
notes_date_br_regex = re.compile(r'\w+\.?\s\d+-?\d*.*,\s(.*)<')
notes_date_br_p_regex = re.compile(r'\w+\.?\s\d+-?\d*<br/>\s(.*)</p>')
notes_comma_regex = re.compile(r'(March|Feb.|February)?\s\d{1,2}-\d{1,2}\s\w*,\s(?P<note>[\w+\s:]*)')
notes_br_regex = re.compile(r'(March|Feb.|February)?\s\d{1,2}-\d{1,2}\s\w*\s*<br/>\s(?P<note>[\w+\s:,/()-.]*)')
notes_no_date_regex = re.compile(r'</strong>[0-9a-zA-Z\s]*,\s(.*)</p>')

excel_columns = ['publish_date', 'category', 'city', 'zip_code', 'street', 'crime_date', 'crime_time', 'add_notes']

##### Data extraction and processing

In [ ]:
def get_parced_data(raw):
    """Returns individual crime records"""
    if raw.find_all("div", {"id": "daily-crime-bulletin"}):
        if raw.find_all("div", {"class": "paragraph--accordion__title"}):
            if raw.find_all("div", {"class": "field__item"}):
                return raw.find_all("div", {"class": "field__item"})

# create list of individual crimes
crimes = [] 
for child in get_parced_data(soup):
    if(child.find('h2')):
        crimes.append(child)

# empty dataframe
crimes_by_date = pd.DataFrame(columns=['publish_date','category','raw_crimes'])

# add crimes at date level
for x in crimes:
    publish_date = x.find("h2").text.split(',',1)[1]
    raw_crimes = x.find_all("p")
    for c in raw_crimes:
        if c.find("u"):
            for cat in c.find_all("u"):
                category = 'cat.text'
                crimes_by_date.loc[len(crimes_by_date.index)] = [publish_date, category, raw_crimes] 

# create empty dataframe with excel columns
by_crime_df = pd.DataFrame(columns=excel_columns)

def by_crime(x):
    """Returns dataframe with extracted attributes for output excel file"""
    for c in x['raw_crimes']:
        # initilize attributes to null
        cat = np.NaN
        street = np.NaN
        crime_date = np.NaN
        crime_time = np.NaN
        add_notes = np.NaN
        
        # Category
        if cat_regex.findall(str(c)):
            cat = cat_regex.findall(str(c))[0]
        # City
        if city_regex.findall(str(c)):
            city = city_regex.findall(str(c))[0]
        # ZipCode
        if zip_regex.findall(str(c)):
            zip_code = zip_regex.findall(str(c))[0]
        # Street 
        if street_regex.findall(str(c)):
            street = street_regex.findall(str(c))[0][0]
        if 'March' in street:
            street = street.split('March')[0]
        if 'Feb.' in street:
            street = street.split('Feb.')[0]
        if 'Feburary' in street:
            street = street.split('Feburary')[0]
        # Date
        if date_regex.findall(str(c)):
            crime_date = date_regex.findall(str(c))[0]
        # Time
        if time_regex.findall(str(c)):
            crime_time = time_regex.findall(str(c))[0]
        # Additional Notes
        if notes_time_regex.findall(str(c)):
            add_notes = notes_time_regex.findall(str(c))[0]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_comma_regex.findall(str(c))) > 0:
            add_notes = notes_comma_regex.findall(str(c))[0][1]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_br_regex.findall(str(c))) > 0:
            add_notes = notes_br_regex.findall(str(c))[0][1]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_time_br_p_regex.findall(str(c))) > 0:
            add_notes = notes_time_br_p_regex.findall(str(c))[0]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_date_br_p_regex.findall(str(c))) > 0:
            add_notes = notes_date_br_p_regex.findall(str(c))[0]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_date_br_regex.findall(str(c))) > 0:
            add_notes = notes_date_br_regex.findall(str(c))[0]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_time_br_br_regex.findall(str(c))) > 0:
            add_notes = notes_time_br_br_regex.findall(str(c))[0]
        if type(add_notes)==float and math.isnan(add_notes) and len(notes_no_date_regex.findall(str(c))) > 0:
            add_notes = notes_no_date_regex.findall(str(c))[0]
        # Add record to dataframe
        by_crime_df.loc[len(by_crime_df.index)] = [x['publish_date'], cat, city, zip_code, street, crime_date, crime_time, add_notes]                    

# Process and extract excel attributes 
crimes_by_date.apply(by_crime, axis = 1)

# Forward fill missing categories 
by_crime_df['category'].fillna(method='ffill',inplace = True)

# process dataframe
by_crime_df.drop_duplicates(inplace = True)
by_crime_df.reset_index(drop=True)

# generate excel 
by_crime_df.to_excel('data/howard-daily-crime-bulletin.xlsx', index=False)